In [7]:
import os
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload

import torch
import torch.nn as nn
import sagemaker
from sagemaker.pytorch import PyTorch

module_path = os.path.abspath(os.path.join('../py-conjugated/'))
if module_path not in sys.path:
    sys.path.append(module_path)
import morphology_networks as net
import model_training as train
import model_testing as test
import physically_informed_loss_functions as PhysLoss
import network_utils as nuts

torch.manual_seed(28)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_bucket = 'sagemaker-us-east-2-362637960691'
train_data_path = 'py-conjugated/m2py_labels/OPV_labels/train_set/'
test_data_path = 'py-conjugated/m2py_labels/OPV_labels/test_set/'
model_states_path = 'py-conjugated/model_states/OPV/'

In [3]:
# sagemaker_session = sagemaker.Session()
# session_bucket = sagemaker_session.default_bucket()
# role = sagemaker.get_execution_role()

# training_image = get_image_uri(sagemaker_session.boto_region_name, 'image-classification', repo_version="latest")

In [4]:
%autoreload

train_dataset = nuts.OPV_ImDataset(data_bucket, train_data_path)
test_dataset = nuts.OPV_ImDataset(data_bucket, test_data_path)

In [9]:
num_epochs = 10
learning_rate = 0.008

In [9]:
# estimator = PyTorch(entry_point = 'train_test_script.py',
#                     source_dir = '../py-conjugated/networks/',
#                     role = role,
#                     framework_version = '1.5.0',
#                     train_instance_count = 1,
#                     train_instance_type = 'ml.p2.xlarge',
#                     hyperparameters = hyperparameters,
#                     sagemaker_session = sagemaker_session)


In [11]:
# estimator.fit(test_data_set, job_name = job_name)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


ValueError: Cannot format input <torch.utils.data.dataloader.DataLoader object at 0x7f6620ed8278>. Expecting one of str, dict, s3_input or FileSystemInput

In [ ]:
# deployed_endpoint = estimator.deploy(
#     initial_instance_count = 1,
#     instance_type = 'ml.t2.medium'
#     update_endpoint = True)

In [ ]:
# for key, im in im_dict.items():
#     print(f'predicting image #: {key}')
    
#     im_tensor = torch.from_numpy(im).float()
#     im_tensor = im_tensor.view(2, 256, 256)
#     im_pred = deployed_endpoint.predict(im_tensor)
    
#     print(f'prediction: {im_pred}')
#     print('----------------------------')

In [ ]:
# deployed_endpoint.delete_endpoint()

In [10]:
model = net.OPV_m2py_NN(2)

#define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [11]:
%autoreload

train_epoch_losses = []
test_epoch_losses = []

for epoch in range(num_epochs):
    
    train_epoch_loss = train.train_OPV_m2py_model(model = model,
                                   training_data_set = train_dataset,
                                   criterion = criterion,
                                   optimizer = optimizer)
    
    train_epoch_losses.append(train_epoch_loss)
    
    test_epoch_loss = test.eval_OPV_m2py_model(model = model,
                                 testing_data_set = test_dataset,
                                 criterion = criterion)
    
    test_epoch_losses.append(test_epoch_loss)

KeyError: 0

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))

epochs = np.arange(1, (num_epochs+1), 1)

plt.plot(epochs, train_epoch_losses, c = 'k', label = 'training error')
plt.plot(epochs, test_epoch_losses, c = 'r', label = 'testing error')
plt.legend(loc = 'upper right')
plt.show()